In [ ]:
import pandas as pd
from aws_helpers import execute_athena_query
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn; seaborn.set()
import matplotlib.dates as mdates
%matplotlib inline

end = datetime.today()
start = end - timedelta(days = 30)

sql = f"""
select * from weather
where observation_ts >= date('{start.strftime('%Y-%m-%d')}')
 and observation_ts < date('{end.strftime('%Y-%m-%d')}')
"""

results_url = execute_athena_query(sql)
df = pd.read_csv(results_url)

df['observation_ts'] = pd.to_datetime(df['observation_ts'])

df.head()

Executing: 
select * from weather
where observation_ts >= date('2021-12-05')
 and observation_ts < date('2022-01-04')

Wait count 0/30
Wait count 1/30
Query succeeded


In [ ]:
f, ax = plt.subplots(figsize=(24, 8))
ax.set_title('UK Temperature last 30 days', fontdict={'fontsize': '18', 'fontweight' : '3'})

temps = df[['observation_ts', 'temperature']]

by_ts = temps.groupby(['observation_ts'])['temperature'].quantile([0.2, 0.5, 0.8]).unstack()

ax.plot(by_ts.index, by_ts[0.5], linewidth=4.0, label="Average temperature")
ax.fill_between(by_ts.index, by_ts[0.2], by_ts[0.8], alpha=0.5)

ax.xaxis.set_major_locator(mdates.DayLocator(interval=1))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%d-%m-%Y'))
plt.gcf().autofmt_xdate()

plt.xlabel('')
plt.ylabel('Temperature', fontsize=12)
plt.legend(loc='lower right')
plt.show()